In [20]:
#import statements
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [21]:
#load spotify dataset
df = pd.read_csv('/content/Dataset/data.csv')
df.head()

serial_num  acousticness  danceability  duration_ms  energy  \
0           0        0.0102         0.833       204600   0.434   
1           1        0.1990         0.743       326933   0.359   
2           2        0.0344         0.838       185707   0.412   
3           3        0.6040         0.494       199413   0.338   
4           4        0.1800         0.678       392893   0.561   

   instrumentalness  key  liveness  loudness  mode  speechiness    tempo  \
0          0.021900    2    0.1650    -8.795     1       0.4310  150.062   
1          0.006110    1    0.1370   -10.401     1       0.0794  160.083   
2          0.000234    2    0.1590    -7.148     1       0.2890   75.044   
3          0.510000    5    0.0922   -15.236     1       0.0261   86.468   
4          0.512000    5    0.4390   -11.648     0       0.0694  174.004   

   time_signature  valence  target      song_title            artist  
0             4.0    0.286       1        Mask Off            Future  
1             4.0    0.588       1         Redbone  Childish Gambino  
2             4.0    0.173       1    Xanny Family            Future  
3             4.0    0.230       1  Master Of None       Beach House  
4             4.0    0.904       1  Parallel Lines       Junior Boys

In [22]:
#Data Description and Insights
df.describe()

serial_num  acousticness  danceability   duration_ms       energy  \
count  2017.000000   2017.000000   2017.000000  2.017000e+03  2017.000000   
mean   1008.000000      0.187590      0.618422  2.463062e+05     0.681577   
std     582.402066      0.259989      0.161029  8.198181e+04     0.210273   
min       0.000000      0.000003      0.122000  1.604200e+04     0.014800   
25%     504.000000      0.009630      0.514000  2.000150e+05     0.563000   
50%    1008.000000      0.063300      0.631000  2.292610e+05     0.715000   
75%    1512.000000      0.265000      0.738000  2.703330e+05     0.846000   
max    2016.000000      0.995000      0.984000  1.004627e+06     0.998000   

       instrumentalness          key     liveness     loudness         mode  \
count       2017.000000  2017.000000  2017.000000  2017.000000  2017.000000   
mean           0.133286     5.342588     0.190844    -7.085624     0.612295   
std            0.273162     3.648240     0.155453     3.761684     0.487347   
min            0.000000     0.000000     0.018800   -33.097000     0.000000   
25%            0.000000     2.000000     0.092300    -8.394000     0.000000   
50%            0.000076     6.000000     0.127000    -6.248000     1.000000   
75%            0.054000     9.000000     0.247000    -4.746000     1.000000   
max            0.976000    11.000000     0.969000    -0.307000     1.000000   

       speechiness        tempo  time_signature      valence       target  
count  2017.000000  2017.000000     2017.000000  2017.000000  2017.000000  
mean      0.092664   121.603272        3.968270     0.496815     0.505702  
std       0.089931    26.685604        0.255853     0.247195     0.500091  
min       0.023100    47.859000        1.000000     0.034800     0.000000  
25%       0.037500   100.189000        4.000000     0.295000     0.000000  
50%       0.054900   121.427000        4.000000     0.492000     1.000000  
75%       0.108000   137.849000        4.000000     0.691000     1.000000  
max       0.816000   219.331000        5.000000     0.992000     1.000000

In [23]:
#Data cleaning
df.isnull().sum()

#No Null values to drop
df = df.drop(['serial_num'], axis=1)
df
print(len(df.artist.unique()))

1343


In [24]:
#Split Data into features and labels
df.columns
feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence',]

X = df[feature_cols]
y = df['target']

In [31]:
#Split Data into Train and Test splits
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25 , random_state = 42)


In [32]:
#Train Logistic Regression Model on new Training data

model = LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)
print(accuracy_score(model.predict(X_train),y_train)) 

0.53505291005291


In [33]:
scores = cross_val_score(model,X_train,y_train)
print(scores.mean())


0.5528948921382205


In [34]:
params = {'C' : np.logspace(-6,3,10)}
clf = GridSearchCV(model,params)
clf.fit(X_train,y_train)
print(clf.best_estimator_.C)
print(clf.best_score_)

0.1
0.5535549581448211


In [35]:
model = LogisticRegression(max_iter = 1000, C = 0.1)
model.fit(X_train,y_train)
predictions = clf.predict(X_test)
print(accuracy_score(predictions,y_test))

0.5485148514851486


In [36]:
print(confusion_matrix(predictions,y_test))
print(classification_report(predictions,y_test))

[[123 110]
 [118 154]]
              precision    recall  f1-score   support

           0       0.51      0.53      0.52       233
           1       0.58      0.57      0.57       272

    accuracy                           0.55       505
   macro avg       0.55      0.55      0.55       505
weighted avg       0.55      0.55      0.55       505



In [37]:
## SVM Model

model_SVM = SVC(kernel='linear')
model_SVM.fit(X_train,y_train)
preds = model_SVM.predict(X_test)
print(accuracy_score(preds,y_test))

0.5663366336633663


In [38]:
print(confusion_matrix(preds,y_test))
print(classification_report(preds,y_test))

[[131 109]
 [110 155]]
              precision    recall  f1-score   support

           0       0.54      0.55      0.54       240
           1       0.59      0.58      0.59       265

    accuracy                           0.57       505
   macro avg       0.57      0.57      0.57       505
weighted avg       0.57      0.57      0.57       505

